This code covers the data processing steps involved in subset of overall 311 data for years and complaint type. 

#### Data Inputs
A csv file is downloaded from the NYC Open Data portal and used as input. Downloaded data is shared in the google docs link shared below to be used for reproducing this code.

NYC Open Data Source: https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9 <br>
Downloaded Data: https://drive.google.com/file/d/0B0DyTS0hhqGockVXNmJRRF9GUUU/view?usp=sharing

The above link contains the source data in zip format which needs to be unzipped before running the below code.

#### Data Outputs
Output data from this code is then used as input for assigning neighborhood level 311 calls made across each school. The final school level aggreagated 311 calls data is then used for the final analysis.

In [1]:
# Required libraries
%pylab inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import seaborn as sns
sns.set(context='notebook', style='whitegrid', palette='deep', font='sans-serif', font_scale=1, rc=None)

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
data311 = pd.read_csv("311_Service_Requests_from_2010_to_Present.csv", parse_dates=['Created Date'])
data311.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [3]:
# Extracting year from date variable
data311_1 = data311.copy()

data311_1.loc[:,'Year'] = data311_1['Created Date'].dt.year
print data311_1.shape
print data311_1.Year.unique()

(609055, 54)
[2016 2015 2014 2013 2012 2011 2010]


In [4]:
# Subset for 2013, 2014, 2015
yearlist = [2013,2014,2015]
data311_2 = data311_1.loc[(data311_1.Year.isin(yearlist)),]
print data311_2.shape
print data311_2.Year.unique()

(313607, 54)
[2015 2014 2013]


In [5]:
data311_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313607 entries, 20401 to 334007
Data columns (total 54 columns):
Unique Key                        313607 non-null int64
Created Date                      313607 non-null datetime64[ns]
Closed Date                       304892 non-null object
Agency                            313607 non-null object
Agency Name                       313607 non-null object
Complaint Type                    313607 non-null object
Descriptor                        313607 non-null object
Location Type                     104268 non-null object
Incident Zip                      286080 non-null object
Incident Address                  192119 non-null object
Street Name                       192118 non-null object
Cross Street 1                    233942 non-null object
Cross Street 2                    233605 non-null object
Intersection Street 1             115212 non-null object
Intersection Street 2             114898 non-null object
Address Type           

In [6]:
# Removing traffic collisions with missing location values
data311_3 = data311_2.copy()
data311_3 = data311_3[pd.notnull(data311_3['Location'])]
data311_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283575 entries, 20401 to 334007
Data columns (total 54 columns):
Unique Key                        283575 non-null int64
Created Date                      283575 non-null datetime64[ns]
Closed Date                       276190 non-null object
Agency                            283575 non-null object
Agency Name                       283575 non-null object
Complaint Type                    283575 non-null object
Descriptor                        283575 non-null object
Location Type                     101855 non-null object
Incident Zip                      283229 non-null object
Incident Address                  176171 non-null object
Street Name                       176171 non-null object
Cross Street 1                    208283 non-null object
Cross Street 2                    208275 non-null object
Intersection Street 1             107696 non-null object
Intersection Street 2             107696 non-null object
Address Type           

In [7]:
# Checking complaint type
print data311_3['Complaint Type'].unique()
print data311_3['Descriptor'].unique()

['Street Condition']
['Pothole' 'Cave-in' 'Defective Hardware' 'Plate Condition - Noisy'
 'Rough, Pitted or Cracked Roads' 'Wear & Tear' 'Line/Marking - Faded'
 'Hummock' 'Line/Marking - After Repaving' 'Failed Street Repair'
 'Blocked - Construction' 'Plate Condition - Open'
 'Plate Condition - Shifted' 'Maintenance Cover' 'Guard Rail - Street'
 'Unsafe Worksite' 'Crash Cushion Defect' 'Suspected Street Cut'
 'Plate Condition - Anti-Skid' 'Depression Maintenance' 'Defacement'
 'General Bad Condition' 'Dumpster - Causing Damage' 'Crack Sealing'
 'Strip Paving' 'Dumpster - Construction Waste']


In [8]:
# Extract the data for years 2015, 2014, 2013 to visualize in CartoDB
data311_3[['Unique Key', 'Latitude', 'Longitude', 'Location']].to_csv('data311Final.csv')

>__"This final data which is aggregated at street qualit complaint level for years 2013, 2014 and 2015 is then used to perform geo-spatial analysis for assigning 311 calls made in each school neighborhood within 300 foot radius."__